In [1]:
import torch
import numpy as np
from pandas import factorize
from Utils.utils import dataset_loader
from uncertain.models.ExplicitFactorizationModel import ExplicitFactorizationModel
from uncertain.models.UncertainWrapper import UncertainWrapper, LinearUncertaintyEstimator
from uncertain.metrics import rmse_score
from evaluation import evaluate_uncertainty

In [2]:
train, test = dataset_loader('1M', 0)
train.gpu()

model = ExplicitFactorizationModel(embedding_dim=50, n_iter=200, batch_size=int(1e6),
                                   learning_rate=0.02, l2=2e-5, use_cuda=True)
model.fit(train)

100%|██████████| 200/200 [00:04<00:00, 45.71it/s, Epoch 200 loss: 0.5849583148956299]


In [3]:
user_support_uncertainty = LinearUncertaintyEstimator(-train.get_user_support(), None)
item_support_uncertainty = LinearUncertaintyEstimator(None, -train.get_item_support())
item_variance_uncertainty = LinearUncertaintyEstimator(None, train.get_item_variance())

In [4]:
user_support = UncertainWrapper(model, user_support_uncertainty)
item_support = UncertainWrapper(model, item_support_uncertainty)
item_variance = UncertainWrapper(model, item_variance_uncertainty)

In [5]:
evaluate_uncertainty(user_support, test, train)

{'RMSE': tensor(0.8503),
 'Precision': array([0.14340411, 0.12469544, 0.11521058, 0.10703098, 0.10010442,
        0.09476157, 0.09062205, 0.08697355, 0.08388444, 0.0811347 ]),
 'Recall': array([0.01706076, 0.02922011, 0.03931335, 0.04860676, 0.05677396,
        0.06472271, 0.07203464, 0.07892997, 0.08503989, 0.09198389]),
 'Correlation': ((0.03790560404744843, 3.9028957102485445e-33),
  SpearmanrResult(correlation=0.0461354267907192, pvalue=2.874071829039173e-48)),
 'RPI': 0.105440825,
 'RRI': array([-0.13455035, -0.18766583, -0.22635532, -0.25938327, -0.28298676,
        -0.30219944, -0.31470497, -0.32845882, -0.34109491, -0.35344093])}

In [6]:
evaluate_uncertainty(item_support, test, train)

{'RMSE': tensor(0.8503),
 'Precision': array([0.14340411, 0.12469544, 0.11521058, 0.10703098, 0.10010442,
        0.09476157, 0.09062205, 0.08697355, 0.08388444, 0.0811347 ]),
 'Recall': array([0.01706076, 0.02922011, 0.03931335, 0.04860676, 0.05677396,
        0.06472271, 0.07203464, 0.07892997, 0.08503989, 0.09198389]),
 'Correlation': ((0.05387214925177378, 3.5057421028997174e-65),
  SpearmanrResult(correlation=0.0510813450243451, pvalue=8.848070782614869e-59)),
 'RPI': 0.14900589,
 'RRI': array([-1.33034718, -0.32226731, -0.40424551, -0.40767654, -0.27247049,
        -0.11014367,  0.10266824,  0.06267181,  0.10352871,  0.07845383])}

In [7]:
evaluate_uncertainty(item_variance, test, train)

{'RMSE': tensor(0.8503),
 'Precision': array([0.14340411, 0.12469544, 0.11521058, 0.10703098, 0.10010442,
        0.09476157, 0.09062205, 0.08697355, 0.08388444, 0.0811347 ]),
 'Recall': array([0.01706076, 0.02922011, 0.03931335, 0.04860676, 0.05677396,
        0.06472271, 0.07203464, 0.07892997, 0.08503989, 0.09198389]),
 'Correlation': ((0.131719086104635, 0.0),
  SpearmanrResult(correlation=0.1140740080587391, pvalue=7.321813950914586e-287)),
 'RPI': 0.4437311,
 'RRI': array([4.07758284, 2.94037138, 1.96382784, 1.70340437, 1.52030399,
        1.40547889, 1.34528273, 1.27010948, 1.20609227, 1.12637134])}